1. Импортируйте библиотеки pandas и numpy. Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn. Создайте датафреймы X и y из этих данных. Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) с помощью функции train_test_split так, чтобы размер тестовой выборки составлял 30% от всех данных, при этом аргумент random_state должен быть равен 42. Создайте модель линейной регрессии под названием lr с помощью класса LinearRegression из модуля sklearn.linear_model. Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых. Вычислите R2 полученных предказаний с помощью r2_score из модуля sklearn.metrics.

https://ru.wikipedia.org/wiki/Обучение_с_учителем

In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_boston

In [10]:
boston = load_boston()
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [61]:
X = pd.DataFrame(boston["data"], columns=boston["feature_names"])
y = pd.DataFrame(boston["target"], columns=["Price"])

In [62]:
from sklearn.model_selection import train_test_split


In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [65]:
from sklearn.linear_model import LinearRegression

https://ru.wikipedia.org/wiki/Линейная_регрессия

In [87]:
?LinearRegression

In [66]:
lr = LinearRegression()

In [67]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [71]:
y_pred = lr.predict(X_test)
y_pred.shape

(152, 1)

In [77]:
check_test = pd.DataFrame({"y_test":y_test["Price"], "y_pred":y_pred.flatten()})
check_test[:5]

,y_test,y_pred
173,23.6,28.648960
274,32.4,36.495014
491,13.6,15.411193
72,22.8,25.403213
452,16.1,18.855280


https://ru.wikipedia.org/wiki/Коэффициент_детерминации
R2 показывает, какую часть изменчивости наблюдаемой переменной можно объяснить с помощью построенной модели, т.е. значение коэффициента детерминации определяет долю (в процентах) изменений, обусловленных влиянием факторных признаков, в общей изменчивости результативного признака.

Значение R2 должно находиться в диапазоне от нуля до единицы: 0 ≤ R2 ≤ 1. Модель считается более качественной, если значение коэффициента детерминации близко к 1. Если R2=1, то эмпирические точки (xi; yi) лежат точно на линии регрессии и между переменными Y и Х существует линейная функциональная зависимость. Если R2=0, то вариация зависимой переменной полностью обусловлена неучтенными в модели факторами.

In [78]:
from sklearn.metrics import r2_score

In [85]:
?r2_score

In [80]:
r2_score(y_test,y_pred)

0.7112260057484925

In [83]:
check_test["error"] = check_test["y_pred"] - check_test["y_test"]

check_test.head()

,y_test,y_pred,error
173,23.6,28.648960,5.048960
274,32.4,36.495014,4.095014
491,13.6,15.411193,1.811193
72,22.8,25.403213,2.603213
452,16.1,18.855280,2.755280


In [84]:
from sklearn.metrics import mean_squared_error

mean_squared_error(check_test["y_pred"], check_test["y_test"])

21.51744423117726

2.Создайте модель под названием model с помощью RandomForestRegressor из модуля sklearn.ensemble. Сделайте агрумент n_estimators равным 1000, max_depth должен быть равен 12 и random_state сделайте равным 42. Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression, но при этом в метод fit вместо датафрейма y_train поставьте y_train.values[:, 0], чтобы получить из датафрейма одномерный массив Numpy, так как для класса RandomForestRegressor в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма.
Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.
Напишите в комментариях к коду, какая модель в данном случае работает лучше.

https://dyakonov.org/2016/11/14/случайный-лес-random-forest/

In [86]:
from sklearn.ensemble import RandomForestRegressor

In [88]:
?RandomForestRegressor

In [89]:
model = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)

Число деревьев — n_estimators

Чем больше деревьев, тем лучше качество, но время настройки и работы RF также пропорционально увеличиваются. Обратите внимание, что часто при увеличении n_estimators качество на обучающей выборке повышается (может даже доходить до 100%), а качество на тесте выходит на асимптоту (можно прикинуть, скольких деревьев Вам достаточно).

(!)Число признаков для выбора расщепления — max_features

График качества на тесте от значения этого праметра унимодальный, на обучении он строго возрастает. При увеличении max_features увеличивается время построения леса, а деревья становятся «более однообразными». По умолчанию он равен sqrt(n) в задачах классификации и n/3 в задачах регрессии. Это самый важный параметр! Его настраивают в первую очередь (при достаточном числе деревьев в лесе).

(!)Максимальная глубина деревьев — max_depth

Ясно, что чем меньше глубина, тем быстрее строится и работает RF. При увеличении глубины резко возрастает качество на обучении, но и на контроле оно, как правило, увеличивается. Рекомендуется использовать максимальную глубину (кроме случаев, когда объектов слишком много и получаются очень глубокие деревья, построение которых занимает значительное время). При использовании неглубоких деревьев изменение параметров, связанных с ограничением числа объектов в листе и для деления, не приводит к значимому эффекту (листья и так получаются «большими»). Неглубокие деревья рекомендуют использовать в задачах с большим числом шумовых объектов (выбросов).

In [91]:
model.fit(X_train, y_train.values[:, 0])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [93]:
y_pred_rfr = model.predict(X_test)
y_pred_rfr.shape

(152,)

In [95]:
check_test_rfr = pd.DataFrame({"y_test":y_test["Price"], "y_pred_rfr":y_pred_rfr.flatten()})
check_test_rfr[:5]

,y_test,y_pred_rfr
173,23.6,22.846138
274,32.4,31.156114
491,13.6,16.297226
72,22.8,23.821036
452,16.1,17.212148


In [96]:
r2_score(y_test,y_pred_rfr)

0.8749965273218174

Much better!

3.Вызовите документацию для класса RandomForestRegressor,
найдите информацию об атрибуте feature_importances. 
С помощью этого атрибута найдите сумму всех показателей важности, установите, какие два признака показывают наибольшую важность.

In [98]:
?RandomForestRegressor

feature_importances_ : array of shape = [n_features]
    The feature importances (the higher, the more important the feature).
  https://towardsdatascience.com/running-random-forests-inspect-the-feature-importances-with-this-code-2b00dd72b92e

In [101]:
feature_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns, 
                                   columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

         importance
LSTAT      0.415679
RM         0.402705
DIS        0.064033
CRIM       0.032117
PTRATIO    0.018090
NOX        0.014368
AGE        0.014245
B          0.012311
TAX        0.011692
INDUS      0.007094
RAD        0.004968
ZN         0.001550
CHAS       0.001149


LSTAT и RM  - черты, которые оказали наибольшее влияние на создание модели предсказания с данным результатом (? верно?)
print(boston["DESCR"])

* 4. В этом задании мы будем работать с датасетом, с которым мы уже знакомы по домашнему заданию по библиотеке Matplotlib, это датасет Credit Card Fraud Detection. Для этого датасета мы будем решать задачу классификации — определять, какие из транзакциции по кредитной карте являются мошенническими.
Данный датасет несбалансирован (так как случаи мошенничества относительно редки), так что применение метрики accuracy не принесет пользы и не поможет выбрать лучшую модель.
Мы будем вычислять AUC, то есть площадь под кривой ROC.

Импорт модулей RandomForestClassifier, GridSearchCV и train_test_split.

In [296]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

 Загрузка датасет creditcard.csv и создание датафрейм df.

In [297]:
url ="/Users/mac/Desktop/Python_for_Data_Science/Lesson4(Matpolotlib)/creditcard.csv"
credit = pd.read_csv(url)


In [298]:
df = pd.DataFrame(credit)

С помощью метода value_counts с аргументом normalize=True убедитесь в том, что выборка несбалансирована.

In [299]:
?pd.value_counts

In [300]:
df['Class'].value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

в бинарной классификации объектов одного из классов меньше 10% (0,17%) - признак несбалансированноести (объектов одного из классов существенно меньше) высока вероятность при обрабоки такой выборки, что данные этого класса не будут участвоавать в расчетах 


Используя метод info, проверьте, все ли столбцы содержат числовые данные и нет ли в них пропусков.


In [301]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

Примените следующую настройку, чтобы можно было просматривать все столбцы датафрейма: pd.options.display.max_columns = 100.
Просмотрите первые 10 строк датафрейма df.

In [302]:
pd.options.display.max_columns = 100

In [303]:
df.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,1.341262,0.359894,-0.358091,-0.137134,0.517617,0.401726,-0.058133,0.068653,-0.033194,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,-0.099254,-1.416907,-0.153826,-0.751063,0.167372,0.050144,-0.443587,0.002821,-0.611987,-0.045575,-0.219633,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,-0.619468,0.291474,1.757964,-1.323865,0.686133,-0.076127,-1.222127,-0.358222,0.324505,-0.156742,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,-0.705117,-0.110452,-0.286254,0.074355,-0.328783,-0.210077,-0.499768,0.118765,0.570328,0.052736,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,1.017614,0.836390,1.006844,-0.443523,0.150219,0.739453,-0.540980,0.476677,0.451773,0.203711,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [304]:
df.keys()

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

Создайте датафрейм X из датафрейма df, исключив столбец Class.


In [305]:
X = df.copy()

In [306]:
X.drop('Class', axis=1, inplace=True)

Создайте объект Series под названием y из столбца Class.


In [308]:
y = pd.Series(df["Class"])

Разбейте X и y на тренировочный и тестовый наборы данных при помощи функции train_test_split, используя аргументы: test_size=0.3, random_state=100, stratify=y.
У вас должны получиться объекты X_train, X_test, y_train и y_test.
Просмотрите информацию о их форме.

In [309]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=100, stratify=y)

In [310]:
N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print(N_train, N_test)
print(X_train.shape)
print(y_train.shape)

199364 85443
(199364, 30)
(199364,)


?GridSearchCV
https://mlbootcamp.ru/article/tutorial/

Для поиска по сетке параметров задайте такие параметры:
parameters = [{'n_estimators': [10, 15], 
'max_features': np.arange(3, 5),
'max_depth': np.arange(4, 7)}]

In [311]:
parameters = [{'n_estimators': [10, 15],
               'max_features': np.arange(3, 5),
               'max_depth': np.arange(4, 7)}]

Создайте модель GridSearchCV со следующими аргументами:
estimator=RandomForestClassifier(random_state=100), 
param_grid=parameters,
scoring='roc_auc',
cv=3.

In [313]:
clf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=100),
    param_grid=parameters,
    scoring='roc_auc',
    cv=3)

Обучение модели на тренировочном наборе данных.

In [314]:
clf.fit(X_train, np.ravel(y_train))

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=100, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [10, 15], 'max_features': array([3, 4]), 'max_depth': array([4, 5, 6])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

Для ознакомления с результатами Grid search использую атрибут .cv_results_, а результаты посмотрим через DataFrame:

In [315]:
cv_results = pd.DataFrame(clf.cv_results_)

cv_results.columns

/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_max_depth', 'param_max_features', 'param_n_estimators', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score',
       'split0_train_score', 'split1_train_score', 'split2_train_score',
       'mean_train_score', 'std_train_score'],
      dtype='object')

Лучшие параметры:

In [316]:
param_columns = [
    column
    for column in cv_results.columns
    if column.startswith('param_')]

score_columns = ['mean_test_score', 'mean_train_score']

cv_results = (cv_results[param_columns + score_columns]
              .sort_values(by=score_columns, ascending=False))

cv_results[:3]

,param_max_depth,param_max_features,param_n_estimators,mean_test_score,mean_train_score
9,6,3,15,0.965969,0.976207
7,5,4,15,0.962598,0.973663
6,5,4,10,0.960201,0.972862


Просмотр параметров лучшей модели с помощью атрибута best_params_.

In [317]:
clf.best_params_

{'max_depth': 6, 'max_features': 3, 'n_estimators': 15}

Соответственно лучшая модель:

In [322]:
clf = RandomForestClassifier(max_depth=6, max_features=3, n_estimators=15)

clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Предскажите вероятности классов с помощью полученнной модели и метода predict_proba.
Из полученного результата (массив Numpy) выберите столбец с индексом 1 (вероятность класса 1) и запишите в массив y_pred_proba.

In [323]:
clf.predict_proba(X_test)

array([[9.98798365e-01, 1.20163495e-03],
       [9.99642328e-01, 3.57672304e-04],
       [9.99757115e-01, 2.42885003e-04],
       ...,
       [9.99594667e-01, 4.05332821e-04],
       [9.98827825e-01, 1.17217465e-03],
       [9.91671229e-01, 8.32877100e-03]])

In [324]:
y_pred_proba = clf.predict_proba(X_test)[:,1]
y_pred_proba

array([0.00120163, 0.00035767, 0.00024289, ..., 0.00040533, 0.00117217,
       0.00832877])

Из модуля sklearn.metrics импортируйте метрику roc_auc_score.
Вычислите AUC на тестовых данных и сравните с результатом, 
полученным на тренировочных данных, используя в качестве аргументов
массивы y_test и y_pred_proba.

In [325]:
from sklearn.metrics import roc_auc_score

In [326]:
roc_auc_score(y_test, y_pred_proba)

0.9464055590850831

ВОПРОС:
    Алгоритм GridSearchCV помогает построить серию моделей (в этом примере рандомфорест) по разным параметрам
    для того чтобы выявить лучшие параметры для этой самой модели (которые можно увидеть например через best_params_)
    Правильно ли что я сразу применил метод roc_auc_score после predict_proba или мне стоило бы запустить рандомфорест с найденными лучшими параметрами, а потом уже roc_auc_score? 
    
   roc_auc_score БЕЗ прогона рандом фореста по лучшим параметрам: 0.9462664156037156
   roc_auc_score С прогоном рандом фореста по лучшим параметрам:0.9464055590850831
 вроде как есть небольшое отличие
 как это работае?

доп материалы:
https://www.swe-notes.ru/post/energy_efficiency/
https://habr.com/ru/post/320726/
https://dyakonov.org/2016/11/14/случайный-лес-random-forest/
https://towardsdatascience.com/running-random-forests-inspect-the-feature-importances-with-this-code-2b00dd72b92e
https://habr.com/ru/company/ods/blog/328372/
https://mlbootcamp.ru/article/tutorial/ (!!)